# Intro to LangMem

This is an early preview of LangMem, a longterm memory service built by LangChain designed help you eaily build personalized user experiences with LLMs. We will walk through the basic functionality as an orientation to the service, including:

1. Creating memory types
2. Posting messages to the service to trigger memory formation
3. Recalling the memories for use in your bot.

My incorporating this in your chat bot, LangMem will asynchronously help it learn their preferences and interests to improve the quality of its responses and recommendations.

## 0. Environment Setup

We've created a demo 'quickstart' instance for you to try out in this notebook.

While LangMem is still in private alpha, you'll need to be given access to a dedicated LangMem instance from a member LangChain team for more personal use beyond this demo. Reach out on Slack or at [support@langchain.dev](mailto:support@langchain.dev) to receive your connection information.

Then you'll want to install the sdk (we will also use openai in our example chat below).

In [5]:
%pip install -U --quiet langmem openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import getpass
import os

# Update to use your instance's URL

os.environ[
    "LANGMEM_API_URL"
] = getpass.getpass("LANGMEM_API_URL (e.g. https://api.langmem.com):")
# Update to your API Key
os.environ["LANGMEM_API_KEY"] = getpass.getpass("API_KEY")
# Used for the example chat bot later in this tutorial
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

With the environment configured, you can create your client. We will connect to OpenAI and to Langmem.

In [10]:
import openai
from langmem import AsyncClient, Client

oai_client = openai.AsyncClient()
langmem_client = AsyncClient()

## 1. Creating custom memory types

Memory types let you model your application domain so your bot can retain inforation in a format appropriate to your use case.

LangMem supports both `user` level and `thread`-level memories.

LangMem supports 3 primary types of `user` memories:

1. Structured "user state" memory to infer and manage a pre-specified user profile
2. Structured "user append state" memory to infer information salient to your application context and query semantically
3. Unstructured user semantic memory

The unstructured semantic memory is enabled by default in each deployment. We will see more of that below.

Enabling *structured* memory functions is as easy as posting a schema to LangMem. LangMem then automatically manages these custom profiles whenever new messages are sent to the service.

### User State

This is a custom user profile. Instantiate by providing a JSON schema or pydantic model.

In [8]:
# %pip install -U --quiet pydantic
import json
from typing import List, Optional
from pydantic import BaseModel, Field


class Course(BaseModel):
    course_name: str = Field(default=None, description="The name of the course.")
    course_id: Optional[str] = Field(default=None, description="An optional course ID.")
    years_teaching: int = Field(default=0, description="Number of years the teacher has been teaching this course.")

class Section(BaseModel):
    period: str = Field(default=None, description="Identifier for the specific time block (e.g., Period 1, Block A).")
    course_name: str = Field(default=None, description="Name of the course being taught in this section.")
    start_time: str = Field(default=None, description="Start time for the class.")
    end_time: str = Field(default=None, description="End time for the class.")
    days: List[str] = Field(default_factory=list, description="Days of the week the class meets.")
    room_number: str = Field(default=None, description="Location of the class.")
    student_count: Optional[int] = Field(default=None, description="Number of students enrolled, if readily available.")

class TeacherProfile(BaseModel):
    name: str = Field(default=None, description="Full name of the teacher.")
    school: str = Field(default=None, description="The name of the school where the teacher is currently employed.")
    email_address: Optional[str] = Field(default=None, description="School email address.")
    phone_number: Optional[str] = Field(default=None, description="Contact number, if necessary.")
    courses_teaching: List[Course] = Field(default_factory=list, description="A list of courses the teacher is currently teaching.")
    total_years_experience: int = Field(default=0, description="Total years of teaching experience.")
    sections: List[Section] = Field(default_factory=list, description="Detailed information about each section the teacher is instructing.")



user_profile_memory = await langmem_client.create_memory_function(
    TeacherProfile, target_type="user_state"
)


### User Append State

As the name suggests, the `user_append_state` is an append-only state (meaning the profile is never overwritten) that lets you define schema(s) to represent individual memories which you can later query semantically.

In [11]:
from pydantic import BaseModel, Field
class ConceptualShift(BaseModel):
    belief: str = Field(
        default="",
        description="A pivotal change in the student's understanding about the world, themselves, or a concept."
    )
    why: str = Field(description="The insight or evidence that triggered the shift in thinking.")
    context: str = Field(
        description="The specific situation or student's words that led to this belief change."
    )

class InsightMoment(BaseModel):
    event: str = Field(
        default="",
        description="The 'aha' moment - a key realization the student had."
    )
    impact: str = Field(default="", description="How this realization might reshape the student's perspective or future thinking.")


### User Semantic Memory

There is also a triplet-based `user_semantic_memory` that is enabled by default. You can turn it off if you don't plan to use it.

In [12]:
functions = langmem_client.list_memory_functions(target_type="user")
semantic_memory = None
async for func in functions:
    if func["type"] == "user_semantic_memory":
        semantic_memory = func


# Uncomment to disable the unstructured memory
# await langmem_client.update_memory_function(semantic_memory["id"], status="disabled")

### Thread Summary

LangMem also supports thread-level memories. We will create them below.

In [13]:
class ConversationSummary(BaseModel):
    title: str = Field(description="Distinct for the conversation.")
    summary: str = Field(description="High level summary of the interactions.")
    topic: List[str] = Field(
        description="Tags for topics discussed in this conversation."
    )


thread_summary_function = await langmem_client.create_memory_function(
    ConversationSummary, target_type="thread_summary"
)

## 2. Starting a conversation

Memories are formed whenever your chat bot posts messages to the service.

Whenever a a user ID is provided in the message metadata, LangMem will automatically create a new user entry and start tracking memories for that user.

In [14]:
import uuid

johnny_user_id = uuid.uuid4()
jimmy_user_id = uuid.uuid4()
jimmy_username = f"jimmy-{uuid.uuid4().hex[:4]}"
johnny_username = f"johnny-{uuid.uuid4().hex[:4]}"

#### The following is an example conversation between 1 or more users and an AI

In [17]:
# Unique for a given converstaion
thread_id = uuid.uuid4()
# from langchain_openai import ChatOpenAI



async def completion(messages: list):
    stripped_messages = [
        {k: v for k, v in m.items() if k != "metadata"} for m in messages
    ]
    completion = await langmem_client.chat.completions.create(
        model="gpt-3.5-turbo", messages=stripped_messages
    )
    return completion


messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {
        "role": "user",
        # Names are optional but should be consistent with a given user id, if provided
        "name": jimmy_username,
        "content": "Hey johnny have i ever told you about my older bro steve?",
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "no, you didn't, but I think he was friends with my younger sister sueann",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "yeah me and him used to play stickball down in the park before school started. I think it was in 1980",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "That was totally 1979! I remember because i was stuck at home all summer.",
        "role": "user",
        "name": "Jeanne",
        # If the user ID isn't provided, we treat this as a guest message and won't assign memories to the user
    },
    {
        "content": "That was so long ago. I have gotten old and gained 200 pounds since then. I can't even remember who was president. @ai, who was the president in 1980?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "The president of the United States in 1980 was Jimmy Carter.",
        "role": "assistant",
    },
    {
        "content": "Wow ya i forgot that. Stickleball really impacted my life. It's how i first met Jeanne! wonder how my life would have turned out if it hadn't happened that way.",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "Yeah wow. That was a big year! @ai could you remind me what else was going on that year?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
]

result = await completion(messages)

messages.append(result.choices[0].message)
print(result.choices[0].message.content)

AttributeError: 'AsyncClient' object has no attribute 'chat'

In [ ]:
# Unique for a given converstaion
thread_id = uuid.uuid4()
# from langchain_openai import ChatOpenAI



async def completion(messages: list):
    stripped_messages = [
        {k: v for k, v in m.items() if k != "metadata"} for m in messages
    ]
    completion = await langmem_client.chat.completions.create(
        model="gpt-3.5-turbo", messages=stripped_messages
    )
    return completion


messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {
        "role": "user",
        # Names are optional but should be consistent with a given user id, if provided
        "name": jimmy_username,
        "content": "Hey johnny have i ever told you about my older bro steve?",
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "no, you didn't, but I think he was friends with my younger sister sueann",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "yeah me and him used to play stickball down in the park before school started. I think it was in 1980",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "That was totally 1979! I remember because i was stuck at home all summer.",
        "role": "user",
        "name": "Jeanne",
        # If the user ID isn't provided, we treat this as a guest message and won't assign memories to the user
    },
    {
        "content": "That was so long ago. I have gotten old and gained 200 pounds since then. I can't even remember who was president. @ai, who was the president in 1980?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "The president of the United States in 1980 was Jimmy Carter.",
        "role": "assistant",
    },
    {
        "content": "Wow ya i forgot that. Stickleball really impacted my life. It's how i first met Jeanne! wonder how my life would have turned out if it hadn't happened that way.",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "Yeah wow. That was a big year! @ai could you remind me what else was going on that year?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
]

result = await completion(messages)

messages.append(result.choices[0].message)
print(result.choices[0].message.content)

AttributeError: 'AsyncClient' object has no attribute 'chat'

In [23]:
messages = [
    {"role": "ai", "content": "Welcome back to your Reflection Journal!"},
    {"role": "ai", "content": "Let's start by revisiting your science lesson today. Do you remember the videos we watched of objects in collisions?"},
    {"role": "human", "name": "CuriousCarl", "content": "Yes! The slow-motion stuff was crazy, especially the golf ball and the glass shattering."},
    {"role": "ai", "content": "What was the most surprising thing you observed?"}, 
    {"role": "human", "name": "CuriousCarl", "content": "Definitely the laser making the glass bend. I didn't think something so hard could change shape like that, even a tiny bit."}
]
messages_2 = [
    {"role": "ai", "content": "Remember, you started the lesson with a claim about how objects react in collisions.  Can you remind me what your initial claim was?"},
    {"role": "human", "name": "CuriousCarl", "content": "I thought that nothing solid would bend or change shape when pushed."},
    {"role": "ai", "content": "Let's think about your observations. Did anything you saw challenge that initial claim?"},
    {"role": "human", "name": "CuriousCarl", "content": "Yeah, pretty much everything! The golf ball, the glass, but especially the concrete beam. That really showed me it's not just about being soft or hard."},
] 
messages_3 = [
    {"role": "ai", "content": "It sounds like this lesson shifted your thinking. Can you summarize what you learned about how objects respond to forces?"},
    {"role": "human", "name": "CuriousCarl", "content": "Even if I can't see it, all solid things must change shape in some way when a force pushes on them.  It's about how much force, and maybe what the thing is made of."},
    {"role": "ai", "content": "Excellent! That's a great insight.  Keep using those observation skills, and don't be afraid to change your ideas when you learn something new!"} 
]
thread_messages = [messages, messages_2, messages_3]
thread_ids = [uuid.uuid4(), uuid.uuid4(), uuid.uuid4()]
thread_dict = dict(zip(thread_ids, thread_messages))



#### Now that we have the messages, we can share them with LangMem.

In [26]:
for id in thread_ids:
    await langmem_client.threads.create(thread_id=id, messages=thread_dict[id])
    
    


AttributeError: 'AsyncClient' object has no attribute 'threads'

In [ ]:
# LangMem will automatically process memories after some delay (~60 seconds), but we can eagerly process the memories as well
await langmem_client.trigger_all_for_thread(thread_id=thread_id)
# You could also trigger for a single user if you'd like
# await langmem_client.trigger_all_for_user(user_id=jimmy_user_id)

#### Fetch messages

You can fetch all the messages in a LangMem thread through that thread's GET endpoint. In this way, LangMem can act as a generic chat bot backend.

In [ ]:
messages = langmem_client.list_messages(thread_id=thread_id)
async for message in messages:
    print(message)

{'id': 'c478e641-dacb-4e3a-8cac-7d9574ed8056', 'content': 'You are a helpful AI assistant', 'timestamp': '2024-03-28T17:10:25.937750', 'user': {'user_id': None, 'user_name': None, 'role': 'system'}}
{'id': 'b0776988-8113-4e2d-b62d-f827eee98867', 'content': 'Hey johnny have i ever told you about my older bro steve?', 'timestamp': '2024-03-28T17:10:25.937784', 'user': {'user_id': 'c1fd3d58-0e67-43ec-b688-1c59f41f79c6', 'user_name': 'jimmy-56c5', 'role': 'user'}}
{'id': '1cd7a340-cebc-495f-8a0d-fb5965c57d40', 'content': "no, you didn't, but I think he was friends with my younger sister sueann", 'timestamp': '2024-03-28T17:10:25.937813', 'user': {'user_id': 'ceab7620-9bd2-45b6-b2e8-88848c0c3965', 'user_name': 'johnny-0fd1', 'role': 'user'}}
{'id': '03c4975c-2656-457e-9851-dd8ee1496216', 'content': 'yeah me and him used to play stickball down in the park before school started. I think it was in 1980', 'timestamp': '2024-03-28T17:10:25.937834', 'user': {'user_id': 'c1fd3d58-0e67-43ec-b688-1c

### 3. Query Memory

You can also query the user memory, once it's updated. This may take a few moments - please be patient 😊

To query the unstructured semantic memory, you can provide query text and the number of memories to return.

In [18]:
# Wait a few moments for the memories to process. If this is empty, you'll likely have to wait a bit longer
mems = None
while not mems:
    mem_response = await langmem_client.query_user_memory(
        user_id=jimmy_user_id, text="stickleball", k=3
    )
    mems = mem_response["memories"]
mems

CancelledError: 

In [ ]:
# In a similar way, you can include
# different `user_append_state` memory results
# in the ranked response
mems = await langmem_client.query_user_memory(
    user_id=jimmy_user_id,
    text="stickleball",
    k=3,
    memory_function_ids=[belief_function["id"], event_function["id"]],
)
mems

{'user_id': 'c1fd3d58-0e67-43ec-b688-1c59f41f79c6',
 'memories': [{'id': '16ad398b-2c3e-461e-9688-ca6cebfa4704',
   'created_at': '2024-03-28T17:10:28.376135Z',
   'last_accessed': '2024-03-28T17:10:44.920281Z',
   'text': '',
   'content': {'event': 'Playing stickball in the park before school in 1980',
    'impact': 'This activity was significant for the user as it was how they first met Jeanne, suggesting it had a profound impact on their life.'},
   'scores': {'recency': 1.0,
    'importance': 0.5,
    'relevance': 0.8382047058016829}},
  {'id': 'a87a5e04-f628-45f0-a6cc-41e35061f013',
   'created_at': '2024-03-28T17:09:46.501123Z',
   'last_accessed': '2024-03-28T17:10:25.875205Z',
   'text': '',
   'content': {'event': 'played stickball in the park with older brother Steve before school started',
    'impact': 'This memory is likely a cherished one, reflecting a close relationship with his brother and a fondness for simpler times.'},
   'scores': {'recency': 0.0, 'importance': 0.5

In [19]:
# For user state (profile) memories, you can make a faster and simple get request
user_state = None
while not user_state:
    user_state = await langmem_client.get_user_memory(
        user_id=jimmy_user_id, memory_function_id=user_profile_memory["id"]
    )
user_state

HTTPError: Client error '404 Not Found' for url 'https://long-term-memory-shared-for-839ac5eb9a19cbc95ef79-vz4y4ooboq-uc.a.run.app/users/47b94ba2-6d0d-4c0d-bf2c-bdce29517c03/memory/5f80aeaf-9ba7-4bff-8e24-8a71ef36fd01/state'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404: {"detail":"User not found"}

In [20]:
# You can list all the thread summary memories for a given thread as well!
await langmem_client.list_thread_memory(thread_id)

HTTPError: Client error '404 Not Found' for url 'https://long-term-memory-shared-for-839ac5eb9a19cbc95ef79-vz4y4ooboq-uc.a.run.app/threads/6fbfad1d-846e-4e81-8d17-4ce62b2804a3/memory'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404: {"detail":"Thread not found"}

## 4. Use in a later conversations

As you can see, we've extracted some useful information from the previous conversation. We imagine you would fetch these facts in later conversations to provide your bot with additional helpful context about the user.

In [ ]:
async def completion_with_memory(messages: list, user_id: uuid.UUID):
    memories = await langmem_client.query_user_memory(
        user_id=user_id, text=messages[-1]["content"], k=3
    )
    facts = "\n".join([mem["text"] for mem in memories["memories"]])
    system_prompt = {
        "role": "system",
        "content": f"You are a helpful assistant. You know the following facts about the user with which you are conversing.\n\n{facts}",
    }
    return await completion([system_prompt] + messages)


messages = [
    {
        "role": "user",
        "name": jimmy_username,
        "content": "Hi there! I'm curious what you remember. What's my brother's name?",
        "metadata": {"user_id": jimmy_user_id},
    }
]
res = await completion_with_memory(messages, user_id=jimmy_user_id)
print(res.choices[0].message.content)

Your older brother's name is Steve.


## Conclusion

In this walkthrough, you saved memories for two users to track their interests and other attributes. You did so simply by sending your chat messages to the LangMem service. You then automatically triggered updates to store long-term memories of three forms:

1. User state "profiles", which follow your custom schema
2. User append state, which store atomic memories following your custom schema and can be queried semantically.
3. General-purpose knowledge as semantic triplets


You also tracked thread-scoped summary memories to help you organize your conversational threads.

Finally, let's clean up our work! This demo is a shared workspace :)

In [21]:
## Cleanup

functions = langmem_client.list_memory_functions()

async for func in functions:
    if func["type"] == "user_semantic_memory":
        continue
    await langmem_client.delete_memory_function(func["id"])